In [ ]:
cd("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior")
import Pkg; 
Pkg.activate(".")

using Plots
using Term
import Term: install_term_logger
import MyterialColors: salmon, green_dark
import KernelDensity: kde
using Statistics
using Colors

using jcontrol
import jcontrol.comparisons: ComparisonPoints

install_term_logger()


In [1]:
# cd("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior")
# import Pkg; 
# Pkg.activate(".")

# # using InfiniteOpt
# import jcontrol.io: load_cached_trials
# import KernelDensity
# KernelDensity.kde(randn(1000))

  Activating project at `~/Documents/Github/LocomotionControl/analysis/behavior`


ArgumentError: ArgumentError: reducing over an empty collection is not allowed

## Figure 3
It shows the speed and angular velocity traces as a function of progression along the track for all trials, with some look at the distribution of values.

In [ ]:
# load trials
trials = load_cached_trials(; keep_n = nothing);
@info "Loaded $(length(trials)) trials"

S = getfield.(trials, :s)
speed = map(t -> sqrt.(t.u .^ 2 .+ t.v .^ 2), trials)
U = getfield.(trials, :u)
Ω = getfield.(trials, :ω);


In [ ]:
# plot as lines plots


function plot_mean_and_std(ŝ, dist, color, subplot)
    μ, σ = dist.μ, dist.σ
    plot!(
        [ŝ, ŝ], [μ - σ, μ + σ], subplot=subplot, lw=4, label=nothing, color="white",
    )
    plot!(
        [ŝ, ŝ], [μ - σ, μ + σ], subplot=subplot, lw=3.5, label=nothing, color=color,
    )
    scatter!(
        [ŝ, ŝ], [μ, μ], subplot=subplot, label=nothing, color=color,
    )


end

# layout
layout = @layout [
    b{0.5h}
    c{0.5h}
]
fig = plot(; layout=layout)

plot!(; xticks=[], ylabel="speed cm/s", subplots=1);
plot!(; xlabel="track progression (cm)", ylabel="angular vel. deg/s", subplots=2)

# plot trials tracking values
plot!.(
    S, U, subplots=1, alpha=.025, color="black", label=nothing
)

plot!(
    S, Ω, subplots=2, alpha=.025, color="black", label=nothing
)

# # plot distributions at comparison points
# for cp in cpoints.points[1:end-1]
#     ŝ = cp.s
#     u = cp.kinematics.u
#     ω = cp.kinematics.ω

#     plot_mean_and_std(ŝ, u, salmon, 1)
#     plot_mean_and_std(ŝ, ω, salmon, 2)

# end



display(fig)

In [ ]:
# load comparison points
cpoints = ComparisonPoints(FULLTRACK; δs=2, trials=trials, s₀=1.0, s₁=250.0);


In [ ]:

function kdeplot(X; scaling=1.0, shift=0.0, v0=0, v1=1.0, bw=1.0, subplot=1.0, color="red")
    _kde = kde(X; npoints=1024, bandwidth=bw)
    # kde = KDEUniv(ContinuousDim(), bw, X, MultiKDE.gaussian)
    # x = Vector(LinRange(v0, v1, granularity_1d))
    # y = [MultiKDE.pdf(kde, _x, keep_all=false) for _x in x]

    plot!(
        _kde.density .* scaling .+ shift,
        _kde.x,    
        label=false, 
        colorbar=false,
        subplot=subplot,   
        color = color,
        line_z=y, 
        lw=2
    )
end

# plot as KDE plots
# layout
layout = @layout [
    b 
    c
]
fig = plot(; layout=layout)


plot!(; ylabel="speed (cm/s)", xticks=[], subplots=1)
plot!(; ylabel="angular vel. (deg/s)",xlabel="track progression (cm)", subplots=2)


# plot!([0, 0], [0, 250], lw=.5, color="black", subplot=2)

# plot distributions at comparison points
for (n, cp) in enumerate(reverse(cpoints.points[1:end-1]))
    ŝ = cp.s
    kdeplot(cp.kinematics.u.values; scaling=150.0, shift=ŝ, v0=0, v1=100, bw=3.5, subplot=1, color=cgrad(:Blues_9, [.05, .2, 1.0]))
    kdeplot(cp.kinematics.ω.values; scaling=10.0, shift=ŝ, v0=-15, v1=15, bw=0.25, subplot=2, color=cgrad(:Blues_9, [.01, .2, 1.0]))
    
end



display(fig)

In [ ]:
cd("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior")
import Pkg; 
Pkg.activate(".")
import KernelDensity: kde
kde(randn(1000))